In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Initialization of plate generation for degradation


Ho importato le librerie che mi servono

In [2]:
import numpy as np
import random
import os

Creo la cartella dove salvare l'inizializzazione degli esperimenti

In [27]:
data_folder = "data_deg"
os.makedirs(data_folder, exist_ok=True)  # crea se non esiste

Definisco le concentrazioni da testare con variabili liste


> Aggiungi citazione




In [4]:
mg_gluta=[6, 7, 8, 9, 10, 11, 12, 13, 14]
peg=[3, 4.5, 6, 7.5]
clpx=[0, 50, 100, 150, 200, 250, 300, 350, 400]
clpp=[0, 50, 100, 150, 200, 250, 300, 350, 400]
atp=[0, 1, 2, 3, 4, 5, 6, 7, 8]
sfgfp=[0, 1.5]
lists=[mg_gluta,peg,clpx,clpp,atp]

Definisco i control samples


In [5]:
ref_deg=[
         mg_gluta[0],
         peg[0],
         clpx[0],
         clpp[0],
         atp[0],
         sfgfp[-1]
]
autofl_deg=[
         mg_gluta[0],
         peg[0],
         clpx[0],
         clpp[0],
         atp[0],
         sfgfp[0]
]

conversione ad array e creazione di quello di controllo

In [6]:
ref_deg=np.reshape(ref_deg,(1,6))
autofl_deg=np.reshape(autofl_deg,(1,6))

ctrl_deg=np.concatenate((ref_deg,autofl_deg),axis=0)

Definisco tutti minimi tranne uno minimo per la degradazione. Sono combinazioni che voglio testare all'inizio

In [7]:
boundary_min_deg=ref_deg
this_sample=[mg_gluta[-1],
         peg[0],
         clpx[0],
         clpp[0],
         atp[0],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_deg=np.concatenate((boundary_min_deg,this_sample),axis=0)

In [8]:
this_sample=[mg_gluta[0],
         peg[-1],
         clpx[0],
         clpp[0],
         atp[0],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_deg=np.concatenate((boundary_min_deg,this_sample),axis=0)


In [9]:
this_sample=[mg_gluta[0],
         peg[0],
         clpx[-1],
         clpp[0],
         atp[0],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_deg=np.concatenate((boundary_min_deg,this_sample),axis=0)

In [10]:
this_sample=[mg_gluta[0],
         peg[0],
         clpx[0],
         clpp[-1],
         atp[0],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_deg=np.concatenate((boundary_min_deg,this_sample),axis=0)


In [11]:
this_sample=[mg_gluta[0],
         peg[0],
         clpx[0],
         clpp[0],
         atp[-1],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_deg=np.concatenate((boundary_min_deg,this_sample),axis=0)


In [12]:
print(boundary_min_deg.shape)

(6, 6)


Ora definisco tutti i massimi tranne uno per la degradazione, compreso quello con tutti i valori al massimo


In [13]:
this_sample=[mg_gluta[-1],
         peg[-1],
         clpx[-1],
         clpp[-1],
         atp[-1],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_deg=this_sample

In [14]:
this_sample=[mg_gluta[0],
         peg[-1],
         clpx[-1],
         clpp[-1],
         atp[-1],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_deg=np.concatenate((boundary_max_deg,this_sample),axis=0)

In [15]:
this_sample=[mg_gluta[-1],
         peg[0],
         clpx[-1],
         clpp[-1],
         atp[-1],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_deg=np.concatenate((boundary_max_deg,this_sample),axis=0)

In [16]:
this_sample=[mg_gluta[-1],
         peg[-1],
         clpx[0],
         clpp[-1],
         atp[-1],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_deg=np.concatenate((boundary_max_deg,this_sample),axis=0)

In [17]:
this_sample=[mg_gluta[-1],
         peg[-1],
         clpx[-1],
         clpp[0],
         atp[-1],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_deg=np.concatenate((boundary_max_deg,this_sample),axis=0)

In [18]:
this_sample=[mg_gluta[-1],
         peg[-1],
         clpx[-1],
         clpp[-1],
         atp[0],
         sfgfp[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_deg=np.concatenate((boundary_max_deg,this_sample),axis=0)


In [19]:
print(boundary_max_deg.shape)

(6, 6)


Ora riempiamo il resto della lista di esperimenti iniziali da fare randomicamente

In [20]:
initial_array_syn=np.concatenate((ctrl_deg,boundary_max_deg,boundary_min_deg[1:]),axis=0) #il primo del boundary_min_deg coincide col reference array
print(initial_array_syn.shape)

(13, 6)


Funzione per vedere se una combinazione è già stata esplorata


In [21]:
def present_in_array_index(new_sample, array):
    """
    Verify if a sample is present in an array.
    """
    if array is None:
        return(False, -1)
    present = False
    new_sample = np.reshape(np.array(new_sample), (1,6))
    for i in range(array.shape[0]):
        if np.array_equiv(array[i,:],new_sample):
            present = True
            break
    return(present, i)

Genero nuovi sample randomici

In [22]:
def generate_random_grid(array_to_avoid, sample_size = 20): #potremmo fare alla prima iterazione 20 e poi le altre di 10 in 10
    active_learning_array = array_to_avoid #array contenente tutte le combinazioni testate e poi quelle che verranno testate
    if array_to_avoid is None:
        answerSize = 0
    else:
        answerSize = active_learning_array.shape[0] #tiene conto delle scelte già fatte. Contatore dei samples già decisi
    # Reagents not allowed to vary
    sfgfp_fixed = sfgfp[1]

    while answerSize < sample_size:
      indexes= tuple(np.random.randint(0, len(lst)) for lst in lists)
      this_sample=[lists[0][indexes[0]],
                  lists[1][indexes[1]],
                  lists[2][indexes[2]],
                  lists[3][indexes[3]],
                  lists[4][indexes[4]],
                  sfgfp_fixed]
      this_sample=np.reshape(this_sample,(1,6))
      if not present_in_array_index(this_sample, active_learning_array)[0]:
        answerSize = answerSize + 1
      if active_learning_array is None:
        active_learning_array = this_sample
      else:
          active_learning_array = np.concatenate((active_learning_array, this_sample), axis = 0)
    return(active_learning_array)

In [23]:
active_learning_array=generate_random_grid(initial_array_syn, sample_size = 15)
print(active_learning_array)

[[  6.    3.    0.    0.    0.    1.5]
 [  6.    3.    0.    0.    0.    0. ]
 [ 14.    7.5 400.  400.    8.    1.5]
 [  6.    7.5 400.  400.    8.    1.5]
 [ 14.    3.  400.  400.    8.    1.5]
 [ 14.    7.5   0.  400.    8.    1.5]
 [ 14.    7.5 400.    0.    8.    1.5]
 [ 14.    7.5 400.  400.    0.    1.5]
 [ 14.    3.    0.    0.    0.    1.5]
 [  6.    7.5   0.    0.    0.    1.5]
 [  6.    3.  400.    0.    0.    1.5]
 [  6.    3.    0.  400.    0.    1.5]
 [  6.    3.    0.    0.    8.    1.5]
 [ 14.    6.  150.  100.    4.    1.5]
 [  9.    3.  100.  250.    8.    1.5]]


In [28]:
headers="mg_gluta; peg8000; ClpX; ClpP; ATP; sfGFP"
np.savetxt(f"{data_folder}/initial_plate_deg_nolabel.csv", active_learning_array, delimiter=";", header=headers, fmt='%.2f', comments="")